In [116]:
#from datasets import load_dataset
#ds = load_dataset("pszemraj/simple_wikipedia")
import torch as t
from tqdm import tqdm
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel


In [6]:
with open('common_words_no_names.txt') as f:
    common_words = f.read().splitlines()
g = t.Generator().manual_seed(214743647)
numerated = {}
for i in range(len(common_words)):
    numerated[common_words[i]] = i

lookup = t.randn((len(common_words), 10), generator = g)
lookup1 = lookup.clone().detach()
common_words = set(common_words)

In [ ]:
for i in tqdm(range(len(ds['train']))):
    paragraph = ds['train'][i]['text'].lower().replace('\n', ' ').split('. ')
    paragraph2 = []
    for p in paragraph:
        if len(p.split('? ')) > 1:
            for g in p.split('? '):
                paragraph2.append(g)
        else:
            paragraph2.append(p)

    sentences = []

    for p in paragraph2:
        if len(p.split('! ')) > 1:
            for g in p.split('! '):
                sentences.append(g.split(' '))
        else:
            sentences.append(p.split(' '))

    for sentence in sentences:
        for word1 in sentence:
            for word2 in sentence:
                if word1 != word2 and word1 in common_words and word2 in common_words and word1 != '' and word2 != '':  # might work better to use all possible words not just common
                    lookup[numerated[word1]] = t.add(lookup[numerated[word1]], 0.05*t.sub(lookup[numerated[word2]], lookup[numerated[word1]]))


In [18]:
def dotNorm(a, b):
    return t.dot(a, b) / (t.norm(a) * t.norm(b))

In [23]:
dotNorm(t.tensor([0,1], dtype=t.float), t.tensor([1,0], dtype=t.float))

tensor(0.)

In [19]:
dotNorm(lookup[numerated['the']], lookup[numerated['cut']])

tensor(1.0000)

In [28]:
lookup[numerated['the']]

tensor([-0.0004,  0.0130,  0.0280,  0.0006,  0.0072,  0.0094, -0.0072, -0.0063,
        -0.0121,  0.0210])

In [30]:
for v in lookup:
    if t.equal(v,lookup[numerated['cat']]):
        print(v)

tensor([-0.0004,  0.0130,  0.0280,  0.0006,  0.0072,  0.0095, -0.0072, -0.0063,
        -0.0121,  0.0210])


In [9]:
changed = 0
count = 0
for i in range(len(common_words)):
    if not lookup1[i].equal(lookup[i]):
        changed += 1
    count += 1

changed / count    

0.9881740262558316

In [77]:
common_words = open('common_words_no_names.txt').read().split()

# add memoization

def anagwams(word):
    if word == '':
        return ['']
    dict = {}
    for w in common_words:
        wordHusk = word
        included = True
        for c in w:
            if c not in wordHusk:
                included = False
                break
            else:
                i = wordHusk.index(c)
                wordHusk = wordHusk[:i] + wordHusk[i+1:]
        if included:
            dict[w] = wordHusk
    return [w + ' ' + g for w in list(dict.keys()) for g in anagwams(dict[w])]

def anagrams(word):
    words = anagwams(word)
    uniqueWords = set([])
    for w in words:
        Wsplit = sorted(sorted(w.split(' ')), key = len)
        uniqueWords.add(str(Wsplit[1:]))
    return list(uniqueWords)

out = anagrams('jonathan')
sorted(out, key = len)

["['joan', 'than']",
 "['an', 'jo', 'than']",
 "['ha', 'tn', 'joan']",
 "['an', 'ha', 'jo', 'tn']"]

In [3]:
model_name = 'gpt2'  # You can use 'gpt2-medium', 'gpt2-large', etc.
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model.eval()

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/Users/JonathanG-B/Desktop/Projects/anagrams/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [127]:
text = "Before the towers"
input_ids = tokenizer.encode(text, return_tensors='pt')
with t.no_grad():
    outputs = model(input_ids)
    logits = outputs.logits

# Get logits for the last token in the input sequence
next_token_logits = logits[:, -1, :]

# Convert logits to probabilities
probabilities = F.softmax(next_token_logits, dim=-1)

# Get the top 10 most probable tokens
top_k = 10
top_k_probs, top_k_indices = t.topk(probabilities, top_k)

top_k_tokens = tokenizer.decode(top_k_indices[0]).split()
top_k_probabilities = top_k_probs[0].tolist()

for token, prob in zip(top_k_tokens, top_k_probabilities):
    print(f"Token: {token}, Probability: {prob:.4f}")

Token: were, Probability: 0.2231
Token: fell, Probability: 0.1174
Token: collapsed, Probability: 0.0579
Token: came, Probability: 0.0547
Token: of, Probability: 0.0371
Token: are, Probability: 0.0366
Token: went,, Probability: 0.0352
Token: opened, Probability: 0.0249
Token: had, Probability: 0.0243


In [128]:
tokens = 'Before the towers'
input_ids = tokenizer.encode(text, return_tensors='pt')
with t.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits

for i in range(list(t.Tensor.size(input_ids))[1] - 1):
    next_token_logits = logits[:, i, :]
    probabilities = F.softmax(next_token_logits, dim=-1)
    next_token_id = input_ids[0][i + 1].item()

    retro_prob = probabilities[0][next_token_id]
    print(retro_prob.item())


0.07800176739692688
2.6427536795381457e-05


In [119]:
t.Tensor.size(probabilities)
probabilities[0]

tensor([1.1324e-06, 1.5012e-06, 2.4056e-08,  ..., 6.6154e-11, 6.8069e-07,
        2.3844e-07])